In [37]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
from IPython.display import display, HTML # used to print out pretty pandas dataframes
import matplotlib.dates as dates
import matplotlib.lines as mlines

pd.options.display.max_colwidth = 500
pd.options.display.width = 500
pd.options.display.max_columns = 500
pd.options.display.max_rows = 10


%matplotlib inline
plt.style.use('ggplot') 

# specify user/password/where the database is
sqluser = 'eightiesfanjan'
sqlpass = 'squiggle'
dbname = 'mimic'
schema_name = 'mimiciii'
host = 'localhost'

query_schema = 'SET search_path to ' + schema_name + ';'

# connect to the database
con = psycopg2.connect(dbname=dbname, user=sqluser, password=sqlpass, host=host)

# Demographic + Admissions Table

subject id, admission id, age, dob, first admit, last discharge, gender, age group, admissions data (mortality, complications)

In [12]:
#get patients whose age between 18-99
#can only get ages less than 89. 

query = query_schema + """

WITH first_admission_time AS
(
  SELECT
      p.subject_id,
      a.hadm_id,
      p.dob, 
      p.gender, 
      MIN (a.admittime) AS first_admittime, 
      MAX(a.dischtime) AS last_discharge,
      MIN( ROUND( (cast(admittime as date) - cast(dob as date)) / 365.242,2) )
          AS first_admit_age
  FROM patients p
  INNER JOIN admissions a
  ON p.subject_id = a.subject_id
  GROUP BY p.subject_id, p.dob, p.gender,a.hadm_id
  ORDER BY p.subject_id
)
SELECT
  first_admittime,
  last_discharge,
  dob,
  first_admit_age, 
  gender,
  CASE
      -- all ages > 89 in the database were replaced with 300
      -- we check using > 100 as a conservative threshold to ensure we capture all these patients
      WHEN first_admit_age > 100
          then '>89'
      WHEN first_admit_age >= 14
          THEN 'adult'
      WHEN first_admit_age <= 1
          THEN 'neonate'
      ELSE 'middle'
      END AS age_group,
  c.*
FROM first_admission_time a
INNER JOIN 
    admissions c
ON a.subject_id = c.subject_id


"""
df_demo= pd.read_sql_query(query,con)
df_demo



,first_admittime,last_discharge,dob,first_admit_age,gender,age_group,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,2191-03-16 00:28:00,2191-03-23 18:41:00,2143-05-12,47.84,F,adult,3,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,None,PROTESTANT QUAKER,SINGLE,WHITE,2191-03-15 13:10:00,2191-03-16 01:10:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",0,1
1,2117-11-20 10:22:00,2117-11-24 14:20:00,2117-11-20,0.00,M,neonate,7,8,159514,2117-11-20 10:22:00,2117-11-24 14:20:00,NaT,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,None,CATHOLIC,None,WHITE,NaT,NaT,NEWBORN,0,1
2,2149-11-09 13:06:00,2149-11-14 10:15:00,2108-01-26,41.79,M,adult,8,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicaid,None,UNOBTAINABLE,None,UNKNOWN/NOT SPECIFIED,2149-11-09 11:13:00,2149-11-09 13:18:00,HEMORRHAGIC CVA,1,1
3,2103-06-28 11:36:00,2103-07-06 12:10:00,2103-06-28,0.00,F,neonate,9,10,184167,2103-06-28 11:36:00,2103-07-06 12:10:00,NaT,NEWBORN,PHYS REFERRAL/NORMAL DELI,SHORT TERM HOSPITAL,Medicaid,None,UNOBTAINABLE,None,BLACK/AFRICAN AMERICAN,NaT,NaT,NEWBORN,0,1
4,2178-04-16 06:18:00,2178-05-11 19:00:00,2128-02-22,50.15,F,adult,10,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,None,OTHER,MARRIED,WHITE,2178-04-15 20:46:00,2178-04-16 06:53:00,BRAIN MASS,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111817,2201-05-15 13:12:00,2201-05-25 14:51:00,2146-10-04,54.61,M,adult,58953,99923,192053,2201-05-15 13:12:00,2201-05-25 14:51:00,NaT,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Private,ENGL,CATHOLIC,MARRIED,WHITE,NaT,NaT,END STAGE LIVER DISEASE,0,1
111818,2201-05-15 13:12:00,2201-05-25 14:51:00,2146-10-04,54.61,M,adult,58952,99923,164914,2201-02-23 20:42:00,2201-03-03 16:00:00,NaT,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,CATHOLIC,MARRIED,WHITE,2201-02-23 15:54:00,2201-02-23 21:58:00,HYPONATREMIA,0,1
111819,2110-02-28 03:47:00,2110-03-06 14:31:00,2092-03-18,17.95,M,adult,58955,99934,176121,2110-02-28 03:47:00,2110-03-06 14:31:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,NaT,NaT,PELVIC ABSCESS,0,1
111820,2128-05-11 15:16:00,2128-05-20 13:35:00,2062-09-06,65.68,F,adult,58958,99937,129380,2128-05-11 15:16:00,2128-05-20 13:35:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,SPAN,CATHOLIC,MARRIED,HISPANIC/LATINO - GUATEMALAN,2128-05-11 11:40:00,2128-05-11 16:45:00,PNEUMONIA,0,1


# Thrombocytopenia Table
Gets subject id, admission id, diagnosis of thrombocytopenia

In [90]:
#3,065 RECORDS
query = query_schema + """

SELECT 
    a.subject_id, 
    a.hadm_id, 
    a.icd9_code
FROM
    diagnoses_icd a
INNER JOIN 
    (
        SELECT 
            icd9_code
        FROM
             d_icd_diagnoses
        WHERE 
            long_title like '%Thrombocytopenia%'
    ) b
ON a.icd9_code = b.icd9_code
ORDER BY subject_id DESC;

"""
df = pd.read_sql_query(query,con)

df


,subject_id,hadm_id,icd9_code
0,99982,183791,2875
1,99982,151454,2875
2,99939,159023,2875
3,99823,179720,2875
4,99802,108099,2875
...,...,...,...
3060,109,102024,2875
3061,109,189332,2875
3062,68,108329,2875
3063,61,176332,2875


# HIT Table
Gets subject id, admission id, for patients with HIT


In [91]:
#97 RECORDS
query = query_schema + """

SELECT 
    DISTINCT(a.hadm_id), 
    a.subject_id,
    a.icd9_code
FROM
    diagnoses_icd a
INNER JOIN 
    (
        SELECT 
            icd9_code
        FROM
             d_icd_diagnoses
        WHERE 
            long_title like '%HIT%'
    ) b
ON a.icd9_code = b.icd9_code

"""
df = pd.read_sql_query(query,con)

df


,hadm_id,subject_id,icd9_code
0,100098,65411,28984
1,100292,62345,28984
2,103246,93595,28984
3,103487,45410,28984
4,104262,79602,28984
...,...,...,...
92,196132,79075,28984
93,196803,82465,28984
94,197499,16856,28984
95,197999,73755,28984


# Hit or Thrombocytopenia with Steroid Use Table 

In [58]:
#1,092 RECORDS
query = query_schema + """


SELECT 
    DISTINCT(a.subject_id), 
    a.hadm_id, 
    a.icd9_code, 
    b.long_title
FROM
    diagnoses_icd a
INNER JOIN 
    (
        SELECT 
            icd9_code, long_title
        FROM
             d_icd_diagnoses
        WHERE 
            long_title like '%HIT%'
    ) b
ON a.icd9_code = b.icd9_code
UNION
    (
    SELECT 
        DISTINCT(a.subject_id), 
        a.hadm_id, 
        a.icd9_code, 
        b.long_title
    FROM
        diagnoses_icd a
    INNER JOIN 
        (
            SELECT 
                icd9_code, long_title
            FROM
                 d_icd_diagnoses
            WHERE 
                long_title like '%Thrombocytopenia%'
        ) b
    ON a.icd9_code = b.icd9_code
    INNER JOIN
        prescriptions d
    ON a.subject_id = d.subject_id
    WHERE
        LOWER(d.drug_name_generic) like '%prednisone%'
            OR 
        LOWER(d.drug) like '%methylprednisolone%'
            OR 
        LOWER(d.drug) like '%dexamethasone%'
    )

"""
df_thrombo_hit = pd.read_sql_query(query,con)

df_thrombo_hit


,subject_id,hadm_id,icd9_code,long_title
0,9402,106278,2875,"Thrombocytopenia, unspecified"
1,10814,155894,2875,"Thrombocytopenia, unspecified"
2,93566,139770,2875,"Thrombocytopenia, unspecified"
3,53019,192310,2875,"Thrombocytopenia, unspecified"
4,69905,111323,2875,"Thrombocytopenia, unspecified"
...,...,...,...,...
1087,109,137510,2875,"Thrombocytopenia, unspecified"
1088,2550,152273,2875,"Thrombocytopenia, unspecified"
1089,24553,152313,2875,"Thrombocytopenia, unspecified"
1090,32453,163483,2875,"Thrombocytopenia, unspecified"


In [59]:
p_demo = pd.merge(df_thrombo_hit,df_demo,on= ['subject_id', 'hadm_id'] )

In [60]:
p_demo.count()

subject_id              3524
hadm_id                 3524
icd9_code               3524
long_title              3524
first_admittime         3524
                        ... 
edregtime               2689
edouttime               2689
diagnosis               3524
hospital_expire_flag    3524
has_chartevents_data    3524
dtype: int64

In [ ]:
p_demo.to_csv('/Users/eightiesfanjan/Desktop/research/mimic3_research/p_demo.csv', index = False)

# Heparin Table
Gets subject id, admission id, heparin date start/stop for patients with thrombocytopenia


In [61]:
query = query_schema + """

SELECT 
    a.subject_id, 
    a.hadm_id, 
    d.*
FROM
    diagnoses_icd a
INNER JOIN 
    (
        SELECT 
            icd9_code
        FROM
             d_icd_diagnoses
        WHERE 
            long_title like '%Thrombocytopenia%'
    ) b
ON a.icd9_code = b.icd9_code
INNER JOIN 
    admissions c
ON a.subject_id = c.subject_id
INNER JOIN
    prescriptions d
ON a.subject_id = d.subject_id
WHERE
    d.drug like '%Heparin%'


"""
df = pd.read_sql_query(query,con)

df


,subject_id,hadm_id,row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route
0,109,172335,1837040,109,166018,241668.0,2141-03-28,2141-03-28,MAIN,Heparin,Heparin,Heparin Sodium,HEPA5I,006549,63323026201,5000 Units / mL- 1mL Vial,5000,UNIT,1,mL,SC
1,109,172335,1837040,109,166018,241668.0,2141-03-28,2141-03-28,MAIN,Heparin,Heparin,Heparin Sodium,HEPA5I,006549,63323026201,5000 Units / mL- 1mL Vial,5000,UNIT,1,mL,SC
2,109,172335,1837040,109,166018,241668.0,2141-03-28,2141-03-28,MAIN,Heparin,Heparin,Heparin Sodium,HEPA5I,006549,63323026201,5000 Units / mL- 1mL Vial,5000,UNIT,1,mL,SC
3,109,172335,1837040,109,166018,241668.0,2141-03-28,2141-03-28,MAIN,Heparin,Heparin,Heparin Sodium,HEPA5I,006549,63323026201,5000 Units / mL- 1mL Vial,5000,UNIT,1,mL,SC
4,109,172335,1837040,109,166018,241668.0,2141-03-28,2141-03-28,MAIN,Heparin,Heparin,Heparin Sodium,HEPA5I,006549,63323026201,5000 Units / mL- 1mL Vial,5000,UNIT,1,mL,SC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123059,95816,156481,3484354,95816,173137,NaN,2191-08-06,2191-08-07,MAIN,Heparin,Heparin,Heparin Sodium,HEPA5I,006549,63323026201,5000 Units / mL- 1mL Vial,5000,UNIT,1,mL,SC
123060,97144,109999,1818023,97144,109999,290949.0,2187-02-08,2187-02-26,MAIN,Heparin,Heparin,Heparin Sodium,HEPA5I,006549,63323026201,5000 Units / mL- 1mL Vial,5000,UNIT,1,mL,SC
123061,97144,109999,1820946,97144,109999,290949.0,2187-02-23,2187-02-26,MAIN,Heparin Flush (10 units/ml),Heparin Flush (10 units/ml),Heparin Flush (10 Units/mL),HEPA10SYR,060304,08290036005,10 Units/mL - 5 mL Syringe,2,mL,0.4,SYR,IV
123062,97144,109999,1820965,97144,109999,290949.0,2187-02-26,2187-02-27,MAIN,Heparin,Heparin,Heparin Sodium,HEPA5I,006549,63323026201,5000 Units / mL- 1mL Vial,5000,UNIT,1,mL,SC


# Argatroban Table
Gets subject id, admission id, argatroban date start/stop for patients with thrombocytopenia


In [62]:

query = query_schema + """

SELECT 
    a.subject_id, 
    a.hadm_id, 
    d.*
FROM
    diagnoses_icd a
INNER JOIN 
    (
        SELECT 
            icd9_code
        FROM
             d_icd_diagnoses
        WHERE 
            long_title like '%Thrombocytopenia%'
    ) b
ON a.icd9_code = b.icd9_code
INNER JOIN 
    admissions c
ON a.subject_id = c.subject_id
INNER JOIN
    prescriptions d
ON a.subject_id = d.subject_id
WHERE
    d.drug like '%Argatroban%'

"""
df = pd.read_sql_query(query,con)

df


,subject_id,hadm_id,row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route
0,1569,138644,1718667,1569,100045,260971.0,2176-02-07,2176-02-08,MAIN,Argatroban,None,None,ARGA250I,047021,00007440701,250mg/2.5mL Vial,250,mg,1,VIAL,IV DRIP
1,1569,138644,1718667,1569,100045,260971.0,2176-02-07,2176-02-08,MAIN,Argatroban,None,None,ARGA250I,047021,00007440701,250mg/2.5mL Vial,250,mg,1,VIAL,IV DRIP
2,1569,138644,1718667,1569,100045,260971.0,2176-02-07,2176-02-08,MAIN,Argatroban,None,None,ARGA250I,047021,00007440701,250mg/2.5mL Vial,250,mg,1,VIAL,IV DRIP
3,1569,138644,1718667,1569,100045,260971.0,2176-02-07,2176-02-08,MAIN,Argatroban,None,None,ARGA250I,047021,00007440701,250mg/2.5mL Vial,250,mg,1,VIAL,IV DRIP
4,1569,138644,1718667,1569,100045,260971.0,2176-02-07,2176-02-08,MAIN,Argatroban,None,None,ARGA250I,047021,00007440701,250mg/2.5mL Vial,250,mg,1,VIAL,IV DRIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,96145,161427,4116956,96145,161427,224187.0,2104-07-22,2104-07-23,MAIN,Argatroban,None,None,ARGA250I,047021,00007440701,250mg/2.5mL Vial,250,mg,1,VIAL,IV DRIP
764,96145,161427,4116956,96145,161427,224187.0,2104-07-22,2104-07-23,MAIN,Argatroban,None,None,ARGA250I,047021,00007440701,250mg/2.5mL Vial,250,mg,1,VIAL,IV DRIP
765,96145,161427,4116960,96145,161427,224187.0,2104-07-23,2104-07-24,MAIN,Argatroban,None,None,ARGA250I,047021,00007440701,250mg/2.5mL Vial,250,mg,1,VIAL,IV DRIP
766,96145,161427,4116960,96145,161427,224187.0,2104-07-23,2104-07-24,MAIN,Argatroban,None,None,ARGA250I,047021,00007440701,250mg/2.5mL Vial,250,mg,1,VIAL,IV DRIP


# Steroid Table
Gets subject id, admission id, steroid date start/stop, for patients with thrombocytopenia


In [95]:

query = query_schema + """

SELECT 
    a.subject_id, 
    a.hadm_id, 
    d.*
FROM
    diagnoses_icd a
INNER JOIN 
    (
        SELECT 
            icd9_code
        FROM
             d_icd_diagnoses
        WHERE 
            long_title like '%Thrombocytopenia%'
    ) b
ON a.icd9_code = b.icd9_code
INNER JOIN 
    admissions c
ON a.subject_id = c.subject_id
INNER JOIN
    prescriptions d
ON a.subject_id = d.subject_id
WHERE
    LOWER(d.drug_name_generic) like '%prednisone%'
        OR 
    LOWER(d.drug) like '%methylprednisolone%' AND route = 'IV'
        OR 
    LOWER(d.drug) like '%dexamethasone%'

"""
df = pd.read_sql_query(query,con)

df


,subject_id,hadm_id,row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route
0,109,172335,1837039,109,166018,241668.0,2141-03-28,2141-03-28,MAIN,PredniSONE,PredniSONE,PredniSONE,PRED5,006753,00054872425,5 mg Tablet,15,mg,3,TAB,PO
1,109,172335,1837039,109,166018,241668.0,2141-03-28,2141-03-28,MAIN,PredniSONE,PredniSONE,PredniSONE,PRED5,006753,00054872425,5 mg Tablet,15,mg,3,TAB,PO
2,109,172335,1837039,109,166018,241668.0,2141-03-28,2141-03-28,MAIN,PredniSONE,PredniSONE,PredniSONE,PRED5,006753,00054872425,5 mg Tablet,15,mg,3,TAB,PO
3,109,172335,1837039,109,166018,241668.0,2141-03-28,2141-03-28,MAIN,PredniSONE,PredniSONE,PredniSONE,PRED5,006753,00054872425,5 mg Tablet,15,mg,3,TAB,PO
4,109,172335,1837039,109,166018,241668.0,2141-03-28,2141-03-28,MAIN,PredniSONE,PredniSONE,PredniSONE,PRED5,006753,00054872425,5 mg Tablet,15,mg,3,TAB,PO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69562,95705,134885,753946,95705,134885,212530.0,2134-12-30,2134-12-31,MAIN,MethylPREDNISolone Sodium Succ,MethylPREDNISolone Sodium Succ,MethylPREDNISolone Sodium Succ,METH40I,051554,00009011319,40mg Vial,80,mg,2,VIAL,IV
69563,95705,134885,753946,95705,134885,212530.0,2134-12-30,2134-12-31,MAIN,MethylPREDNISolone Sodium Succ,MethylPREDNISolone Sodium Succ,MethylPREDNISolone Sodium Succ,METH40I,051554,00009011319,40mg Vial,80,mg,2,VIAL,IV
69564,97263,164631,3786883,97263,164631,223670.0,2122-01-16,2122-01-21,MAIN,Dexamethasone,Dexamethasone,Dexamethasone Sod Phosphate,DEXA4I,006778,63323016501,4mg/mL 1mL Vial,4,mg,1,VIAL,IV
69565,99939,159023,3394229,99939,159023,252442.0,2110-03-22,2110-03-23,MAIN,MethylPREDNISolone Sodium Succ,None,None,MPRED5I,051556,00009076502,500mg Vial,500,mg,1,VIAL,IV


# HIT Tables

- All tables below, unless otherwise noted, are specific to patients with HIT or Thrombocytopenia
- Each table's link provides relevant documentation 

## Get table of procedures for each patient

https://mimic.physionet.org/mimictables/procedureevents_mv/

In [30]:

query = query_schema + """

SELECT
   *
FROM
    procedureevents_mv a
INNER JOIN
    d_items b
ON a.itemid = b.itemid

"""
df_proc = pd.read_sql_query(query,con)

df_proc


,row_id,subject_id,hadm_id,icustay_id,starttime,endtime,itemid,value,valueuom,location,locationcategory,storetime,cgid,orderid,linkorderid,ordercategoryname,secondaryordercategoryname,ordercategorydescription,isopenbag,continueinnextdept,cancelreason,statusdescription,comments_editedby,comments_canceledby,comments_date,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
0,379,29070,115071,232563.0,2145-03-12 23:04:00,2145-03-12 23:05:00,225401,1.0,None,None,None,2145-03-12 23:04:00,20855,5092605,5092605,Procedures,None,Electrolytes,0,0,0,FinishedRunning,None,None,None,13826,225401,Blood Cultured,Blood Cultured,metavision,procedureevents_mv,6-Cultures,None,Process,None
1,380,29070,115071,232563.0,2145-03-12 23:04:00,2145-03-12 23:05:00,225454,1.0,None,None,None,2145-03-12 23:04:00,20855,2538599,2538599,Procedures,None,Electrolytes,0,0,0,FinishedRunning,None,None,None,13916,225454,Urine Culture,Urine Culture,metavision,procedureevents_mv,6-Cultures,None,Process,None
2,381,29070,115071,232563.0,2145-03-12 23:05:00,2145-03-18 20:01:00,225792,8456.0,hour,None,None,2145-03-18 20:01:39,20889,2383835,2383835,Ventilation,None,Task,1,0,0,FinishedRunning,None,None,None,13977,225792,Invasive Ventilation,Invasive Ventilation,metavision,procedureevents_mv,2-Ventilation,None,Process,None
3,382,29070,115071,232563.0,2145-03-12 23:36:00,2145-03-12 23:37:00,225402,1.0,None,None,None,2145-03-12 23:36:00,20855,9355062,9355062,Procedures,None,Electrolytes,0,0,0,FinishedRunning,None,None,None,13827,225402,EKG,EKG,metavision,procedureevents_mv,4-Procedures,None,Process,None
4,383,29070,115071,232563.0,2145-03-13 01:27:00,2145-03-16 16:00:00,224560,5193.0,min,Right IJ,Invasive Venous,2145-03-16 17:57:00,20855,466169,466169,Invasive Lines,None,Task,1,0,0,FinishedRunning,None,None,None,13396,224560,PA Catheter,PA Catheter,metavision,procedureevents_mv,Access Lines - Invasive,None,Process,None
5,384,29070,115071,232563.0,2145-03-13 08:45:00,2145-03-13 08:46:00,225439,1.0,None,None,None,2145-03-13 08:45:00,17525,1909827,1909827,Procedures,None,Electrolytes,0,0,2,Rewritten,RN,None,2145-03-13 10:16:00,13903,225439,Endoscopy,Endoscopy,metavision,procedureevents_mv,4-Procedures,None,Process,None
6,385,29070,115071,232563.0,2145-03-13 09:00:00,2145-03-13 09:01:00,225439,1.0,None,None,None,2145-03-13 10:16:00,17525,3485753,3485753,Procedures,None,Electrolytes,0,0,0,FinishedRunning,None,None,None,13903,225439,Endoscopy,Endoscopy,metavision,procedureevents_mv,4-Procedures,None,Process,None
7,386,29070,115071,232563.0,2145-03-14 09:00:00,2145-03-14 09:01:00,225402,1.0,None,None,None,2145-03-14 10:30:00,17525,8204763,8204763,Procedures,None,Electrolytes,0,0,0,FinishedRunning,None,None,None,13827,225402,EKG,EKG,metavision,procedureevents_mv,4-Procedures,None,Process,None
8,387,29070,115071,232563.0,2145-03-14 10:30:00,2145-03-14 10:31:00,225402,1.0,None,None,None,2145-03-14 10:30:00,17525,1572834,1572834,Procedures,None,Electrolytes,0,0,2,Rewritten,RN,None,2145-03-14 10:30:00,13827,225402,EKG,EKG,metavision,procedureevents_mv,4-Procedures,None,Process,None
9,388,29070,115071,232563.0,2145-03-15 06:26:00,2145-03-15 06:27:00,225459,1.0,None,None,None,2145-03-15 06:26:00,17248,5539417,5539417,Imaging,None,Electrolytes,0,0,0,FinishedRunning,None,None,None,13919,225459,Chest X-Ray,Chest X-Ray,metavision,procedureevents_mv,5-Imaging,None,Process,None


In [ ]:
p_proc = pd.merge(df_thrombo_hit,p_proc,on= ['subject_id', 'hadm_id'] )

In [32]:
p_proc.count()

subject_id             12802
hadm_id                12802
icd9_code              12802
long_title             12802
row_id                 12802
icustay_id             12797
starttime              12802
endtime                12802
itemid                 12802
value                  12802
valueuom               12802
location                2170
                       ...  
comments_canceledby      374
comments_date            539
row_id                 12802
itemid                 12802
label                  12802
abbreviation           12802
dbsource               12802
linksto                12802
category               12802
unitname               12802
param_type             12802
conceptid                  0
dtype: int64

In [54]:
p_proc.to_csv('/Users/eightiesfanjan/Desktop/research/mimic3_research/p_proc.csv', index = False)

## Get table of microbiology events

https://mimic.physionet.org/mimictables/microbiologyevents/

In [49]:

query = query_schema + """


SELECT 
    a.*, 
    b.label, 
    c.label
FROM
    microbiologyevents a
INNER JOIN
    d_items b
ON a.ab_itemid = b.itemid
INNER JOIN
    d_items c
ON a.spec_itemid = c.itemid

"""
df_micro = pd.read_sql_query(query,con)

df_micro


,row_id,subject_id,hadm_id,chartdate,charttime,spec_itemid,spec_type_desc,org_itemid,org_name,isolate_num,ab_itemid,ab_name,dilution_text,dilution_comparison,dilution_value,interpretation,label,label
0,230901,17417,158511,2102-11-12,2102-11-12 07:23:00,70091,MRSA SCREEN,80023.0,STAPH AUREUS COAG +,1,90016,OXACILLIN,None,None,NaN,R,OXACILLIN,MRSA SCREEN
1,230914,17417,158511,2102-11-21,2102-11-21 18:15:00,70076,TISSUE,80053.0,ENTEROCOCCUS SP.,1,90025,LEVOFLOXACIN,=>8,=>,8.0,R,LEVOFLOXACIN,TISSUE
2,230915,17417,158511,2102-11-21,2102-11-21 18:15:00,70076,TISSUE,80053.0,ENTEROCOCCUS SP.,1,90004,AMPICILLIN,<=2,<=,2.0,S,AMPICILLIN,TISSUE
3,230916,17417,158511,2102-11-21,2102-11-21 18:15:00,70076,TISSUE,80053.0,ENTEROCOCCUS SP.,1,90002,PENICILLIN,8,=,8.0,S,PENICILLIN,TISSUE
4,230917,17417,158511,2102-11-21,2102-11-21 18:15:00,70076,TISSUE,80023.0,STAPH AUREUS COAG +,1,90007,CLINDAMYCIN,=>8,=>,8.0,R,CLINDAMYCIN,TISSUE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275761,230894,17417,158511,2102-11-07,2102-11-07 17:07:00,70076,TISSUE,80002.0,ESCHERICHIA COLI,1,90021,PIPERACILLIN,64,=,64.0,I,PIPERACILLIN,TISSUE
275762,230895,17417,158511,2102-11-07,2102-11-07 17:07:00,70076,TISSUE,80002.0,ESCHERICHIA COLI,1,90022,AMPICILLIN/SULBACTAM,=>32,=>,32.0,R,AMPICILLIN/SULBACTAM,TISSUE
275763,230896,17417,158511,2102-11-07,2102-11-07 17:07:00,70076,TISSUE,80002.0,ESCHERICHIA COLI,1,90025,LEVOFLOXACIN,<=0.25,<=,0.0,S,LEVOFLOXACIN,TISSUE
275764,230897,17417,158511,2102-11-07,2102-11-07 17:07:00,70076,TISSUE,80002.0,ESCHERICHIA COLI,1,90029,MEROPENEM,<=0.25,<=,0.0,S,MEROPENEM,TISSUE


In [50]:
p_micro = pd.merge(df_thrombo_hit,df_micro,on= ['subject_id', 'hadm_id'] )

In [52]:
p_micro.count()

subject_id             10636
hadm_id                10636
icd9_code              10636
long_title             10636
row_id                 10636
                       ...  
dilution_comparison    10172
dilution_value         10172
interpretation         10636
label                  10636
label                  10636
dtype: int64

In [53]:
p_micro.to_csv('/Users/eightiesfanjan/Desktop/research/mimic3_research/p_micro.csv', index = False)

## Get lab data

https://mimic.physionet.org/mimictables/labevents/

NOTE: Patient data does not include Pf4 antibody assay nor HIT panel

In [28]:
#Attempt at finding pf4 antibody assay. Not found : (
query = query_schema + """


SELECT 
    DISTINCT(label)
FROM
    labevents a
INNER JOIN
    d_labitems b
ON a.itemid = b.itemid
WHERE lower(label) like '%antibody%' or lower(label) like '%panel%'
"""
df = pd.read_sql_query(query,con)

df


,label
0,"Anti-Gliadin Antibody, IgA"
1,Anti-Mitochondrial Antibody
2,Anti-Neutrophil Cytoplasmic Antibody
3,Anti-Nuclear Antibody
4,"Anti-Nuclear Antibody, Titer"
5,Anti-Parietal Cell Antibody
6,Anti-Smooth Muscle Antibody
7,Anticardiolipin Antibody IgG
8,Anticardiolipin Antibody IgM
9,HIV Antibody


In [56]:

query = query_schema + """


SELECT 
    *
FROM
    labevents a
INNER JOIN
    d_labitems b
ON a.itemid = b.itemid
WHERE lower(label) like '%heparin%'
"""
df_labs = pd.read_sql_query(query,con)

df_labs


,row_id,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,flag,row_id,itemid,label,fluid,category,loinc_code
0,225704,356,192219.0,51228,2132-09-06 04:53:00,0.83,0.83,U/mL,abnormal,428,51228,Heparin,Blood,Hematology,13055-9
1,649590,984,152912.0,51228,2143-02-11 16:39:00,LESS THAN 0.1,NaN,U/mL,None,428,51228,Heparin,Blood,Hematology,13055-9
2,1077221,1709,176360.0,51228,2116-03-04 19:21:00,0.17,0.17,U/mL,abnormal,428,51228,Heparin,Blood,Hematology,13055-9
3,1077263,1709,176360.0,51228,2116-03-05 02:04:00,0.13,0.13,U/mL,abnormal,428,51228,Heparin,Blood,Hematology,13055-9
4,1081977,1709,127294.0,51228,2118-01-09 20:45:00,0.20,0.20,U/mL,abnormal,428,51228,Heparin,Blood,Hematology,13055-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,27059745,93640,175706.0,51229,2166-08-18 10:55:00,0.60,0.60,U/mL,None,429,51229,"Heparin, LMW",Blood,Hematology,32684-3
830,27470759,96741,138697.0,51229,2129-05-10 17:10:00,0.04,0.04,U/mL,None,429,51229,"Heparin, LMW",Blood,Hematology,32684-3
831,27561502,97497,168949.0,51229,2106-07-18 08:20:00,0.27,0.27,U/mL,None,429,51229,"Heparin, LMW",Blood,Hematology,32684-3
832,27561542,97497,168949.0,51229,2106-07-19 09:50:00,0.41,0.41,U/mL,None,429,51229,"Heparin, LMW",Blood,Hematology,32684-3


In [97]:
p_labs = pd.merge(df_thrombo_hit,df_labs,on= ['subject_id', 'hadm_id'] )

In [102]:
p_labs.count()

subject_id    17
hadm_id       17
icd9_code     17
long_title    17
row_id        17
itemid        17
charttime     17
value         17
valuenum      16
valueuom      17
flag           3
row_id        17
itemid        17
label         17
fluid         17
category      17
loinc_code    17
dtype: int64

In [120]:
p_labs.to_csv('/Users/eightiesfanjan/Desktop/research/mimic3_research/p_labs.csv', index = False)

## Get chart data 

https://mimic.physionet.org/mimictables/chartevents/

NOTE: Due to hardware constraints, this data serves only as a sample. The subject_id and hadm_id are not specifically those with HIT or Thrombo.

In [16]:

query = query_schema + """

SELECT 
    *
FROM
    chartevents a
INNER JOIN
    d_items c
ON a.itemid = c.itemid
INNER JOIN
    caregivers b
ON a.cgid = b.cgid
LIMIT 1000


"""
df_charts = pd.read_sql_query(query,con)

df_charts


,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid,row_id,cgid,label,description
0,154784925,23947,160236,214606,113,2170-10-28 21:00:00,2170-10-29 02:52:00,20034,14,14.0,mmHg,None,None,None,NotStopd,118,113,CVP,None,carevue,chartevents,None,None,None,None,6081,20034,RN,RN
1,154784926,23947,160236,214606,128,2170-10-28 21:00:00,2170-10-28 21:46:00,20034,Full Code,NaN,None,None,None,None,NotStopd,131,128,Code Status,None,carevue,chartevents,None,None,None,None,6081,20034,RN,RN
2,154784927,23947,160236,214606,159,2170-10-28 21:00:00,2170-10-28 21:46:00,20034,None,NaN,None,None,None,None,NotStopd,160,159,Ectopy Frequency,None,carevue,chartevents,None,None,None,None,6081,20034,RN,RN
3,154784928,23947,160236,214606,161,2170-10-28 21:00:00,2170-10-28 21:46:00,20034,None,NaN,None,None,None,None,NotStopd,162,161,Ectopy Type,None,carevue,chartevents,None,None,None,None,6081,20034,RN,RN
4,154784938,23947,160236,214606,31,2170-10-28 22:00:00,2170-10-28 21:46:00,20034,Bedrest,NaN,None,None,None,None,NotStopd,39,31,Activity,None,carevue,chartevents,None,None,None,None,6081,20034,RN,RN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,154660521,23905,145496,256668,52,2133-07-09 13:00:00,2133-07-09 13:26:00,14562,100,100.0,mmHg,None,None,None,NotStopd,58,52,Arterial BP Mean,None,carevue,chartevents,None,None,None,None,631,14562,RN,None
996,154660522,23905,145496,256668,127,2133-07-09 13:00:00,2133-07-09 13:26:00,14562,Circulation Adeq,NaN,None,None,None,None,NotStopd,130,127,Circulation/SkinInt,None,carevue,chartevents,None,None,None,None,631,14562,RN,None
997,154660523,23905,145496,256668,128,2133-07-09 13:00:00,2133-07-09 13:26:00,14562,Full Code,NaN,None,None,None,None,NotStopd,131,128,Code Status,None,carevue,chartevents,None,None,None,None,631,14562,RN,None
998,154660524,23905,145496,256668,159,2133-07-09 13:00:00,2133-07-09 13:26:00,14562,None,NaN,None,None,None,None,NotStopd,160,159,Ectopy Frequency,None,carevue,chartevents,None,None,None,None,631,14562,RN,None


In [6]:
df_charts.to_csv('/Users/eightiesfanjan/Desktop/research/mimic3_research/p_charts.csv', index = False)

## Get note events
https://mimic.physionet.org/mimictables/noteevents/

In [15]:
query = query_schema + """

SELECT 
    c.*
FROM
    diagnoses_icd a
INNER JOIN 
    (
        SELECT 
            icd9_code
        FROM
             d_icd_diagnoses
        WHERE 
            long_title like '%HIT%'
    ) b
ON a.icd9_code = b.icd9_code
INNER JOIN
    noteevents c
ON a.subject_id = c.subject_id AND a.hadm_id = c.hadm_id

"""
df_notes = pd.read_sql_query(query,con)

df_notes


,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,8205,402,167615,2156-11-18,None,None,Discharge summary,Report,NaN,None,Admission Date: [**2156-11-11**] Discharge Date: [**2156-11-18**]\n\nDate of Birth: [**2105-9-26**] Sex: F\n\nService: MEDICINE\n\nAllergies:\nPenicillins / Tetracyclines / Plaquenil / Chloroquine /\nSulfonamides / Floxin / Heparin Agents\n\nAttending:[**First Name3 (LF) 2009**]\nChief Complaint:\nHypotension\n\nMajor Surgical or Invasive Procedure:\nTransesophageal [**First Name3 (LF) **]\nPICC line placement\n\n\nHistory of Present Illness:\nThe pt is a 51-ye...
1,66774,402,167615,2156-11-17,None,None,Echo,Report,NaN,None,PATIENT/TEST INFORMATION:\nIndication: Endocarditis.\nHeight: (in) 65\nWeight (lb): 195\nBSA (m2): 1.96 m2\nBP (mm Hg): 110/75\nHR (bpm): 85\nStatus: Inpatient\nDate/Time: [**2156-11-17**] at 13:34\nTest: TEE (Congenital)\nDoppler: Full Doppler and color Doppler\nContrast: None\nTechnical Quality: Adequate\n\n\nINTERPRETATION:\n\nFindings:\n\nLEFT ATRIUM: Mild LA enlargement. Good (>20 cm/s) LAA ejection velocity. All\nfour pulmonary veins not identified.\n\nRIGHT ATRIUM/INTERATRIAL SEPTUM: ...
2,66775,402,167615,2156-11-15,None,None,Echo,Report,NaN,None,PATIENT/TEST INFORMATION:\nIndication: Endocarditis.\nHeight: (in) 66\nWeight (lb): 195\nBSA (m2): 1.98 m2\nBP (mm Hg): 109/68\nHR (bpm): 109\nStatus: Inpatient\nDate/Time: [**2156-11-15**] at 13:01\nTest: Portable TTE (Complete)\nDoppler: Full Doppler and color Doppler\nContrast: None\nTechnical Quality: Adequate\n\n\nINTERPRETATION:\n\nFindings:\n\nThis study was compared to the prior study of [**2156-9-6**].\n\n\nLEFT ATRIUM: Normal LA and RA cavity sizes.\n\nLEFT VENTRICLE: Normal LV wal...
3,145592,402,167615,2156-11-11,None,None,ECG,Report,NaN,None,Sinus tachycardia. Right bundle-branch block. Compared to the previous\ntracing of [**2156-11-9**] the rate is slightly increased.\n\n
4,145593,402,167615,2156-11-09,None,None,ECG,Report,NaN,None,Sinus rhythm. Consider left atrial abnormality. Borderline P-R interval\nprolongation. Right inferior axis with Q waves in leads III and aVF.\nLeft posterior hemiblock versus inferior myocardial infarction and\nright bundle-branch block. Since the previous tracing of [**2155-5-23**] the\naxis is similar but inferior Q waves are more prominent and the QRS complex\nis wider. ST-T wave abnormalities are more prominent. Clinical correlation is\nsuggested.\n\n
...,...,...,...,...,...,...,...,...,...,...,...
8076,1228468,96928,122771,2139-01-02,2139-01-02 15:07:00,None,Radiology,P ART DUP EXT UP UNI OR LMTD PORT,NaN,None,[**2139-1-2**] 3:07 PM\n ART DUP EXT UP UNI OR LMTD PORT Clip # [**Clip Number (Radiology) 56173**]\n Reason: eval for arterial flow\n Admitting Diagnosis: ATHEROSCLEROSIS OF EXTREMITY ANGIOGRAM\n ______________________________________________________________________________\n [**Hospital 2**] MEDICAL CONDITION:\n 65 year old woman vasculopath who had a L brachial a line (now removed) with\n decreased signal by doppler in radial artery\n REASON FOR THIS EXAM...
8077,1229343,96928,122771,2139-01-08,2139-01-08 20:22:00,None,Radiology,PORTABLE ABDOMEN,NaN,None,[**2139-1-8**] 8:22 PM\n PORTABLE ABDOMEN Clip # [**Clip Number (Radiology) 57047**]\n Reason: evaluate if postpyloric. get low CXR/high KUB\n Admitting Diagnosis: ATHEROSCLEROSIS OF EXTREMITY ANGIOGRAM\n ______________________________________________________________________________\n [**Hospital 2**] MEDICAL CONDITION:\n 65 year old woman s/p dobhoff nasointestinal tube\n REASON FOR THIS EXAMINATION:\n evaluate if postpyloric. get low CXR/hig...
8078,1229218,96928,122771,2139-01-08,2139-01-08 07:32:00,None,Radiology,CHEST (PORTABLE AP),NaN,None,[**2139-1-8**] 7:32 AM\n CHEST (PORTABLE AP) Clip # [**Clip Number (Radiology) 56865**]\n Reason: check if postpyloric\n Admitting Diagnosis: ATHEROSCLEROSIS OF EXTREMITY ANGIOGRAM\n _____________________________________________________________

In [95]:
p_notes = pd.merge(df_thrombo_hit,df_notes,on= ['subject_id', 'hadm_id'] )

In [96]:
p_notes.count()

subject_id     8088
hadm_id        8088
icd9_code      8088
long_title     8088
row_id         8088
chartdate      8088
charttime      7459
storetime      6007
category       8088
description    8088
cgid           6007
iserror          19
text           8088
dtype: int64

In [121]:
p_notes.to_csv('/Users/eightiesfanjan/Desktop/research/mimic3_research/p_notes.csv', index = False)